In [1]:
import pandas as pd
import numpy as np

# state level profiles from EFS

In [ ]:
wecc_regions = pd.read_csv('../InputData/wecc_counties_population.csv', index_col=0)
df = pd.read_csv('../InputData/EFSLoadProfile_Reference_Moderate.csv')

In [11]:
df['Year'].unique()

array([2018, 2020, 2024, 2030, 2040, 2050])

In [12]:
efs_loads = pd.DataFrame({'Hour':np.arange(1, 8761)})
for state in ['AZ', 'CA', 'CO', 'ID', 'MT', 'NE', 'NM', 'ND', 'NV', 'OK', 'OR', 'SD', 'TX', 'UT', 'WA', 'WY']:
    for year in [2018, 2030, 2040]:
#     for year in [2018, 2020, 2024, 2030, 2040, 2050]:
        local_load = df.loc[(df['Year']==year)&(df['State']==state)&(~(df['Subsector']=='light-duty vehicles'))].groupby(by='LocalHourID').sum()['LoadMW']
        efs_loads[state+'_'+str(year)+'_nonLDV'] = local_load.values
        local_load = df.loc[(df['Year']==year)&(df['State']==state)&((df['Subsector']=='light-duty vehicles'))].groupby(by='LocalHourID').sum()['LoadMW']
        efs_loads[state+'_'+str(year)+'_LDV'] = local_load.values

# convert from state level to wecc level

In [15]:
wecc_regions

,State_name,Geo_FIPS,County_name,Population,ID,Region_name
0,Arizona,4001,Apache County,71522,28503,Region_NW
1,Arizona,4001,Apache County,71522,25471,Region_SW
2,Arizona,4003,Cochise County,126279,25471,Region_SW
3,Arizona,4005,Coconino County,140217,25471,Region_SW
4,Arizona,4005,Coconino County,140217,28503,Region_NW
...,...,...,...,...,...,...
529,Wyoming,56037,Sweetwater County,44117,28503,Region_NW
530,Wyoming,56039,Teton County,23059,28503,Region_NW
531,Wyoming,56041,Uinta County,20609,28503,Region_NW
532,Wyoming,56043,Washakie County,8129,28503,Region_NW


In [17]:
regions1 = pd.read_csv('../InputData/county_eiaregion_population_mapping.csv', index_col=0)

In [18]:
states = np.sort(list(set(wecc_regions['State_name'])))
state_shortforms = {'Arizona':'AZ', 'California':'CA', 'Colorado':'CO', 'Idaho':'ID', 'Montana':'MT', 
                    'Nebraska':'NE', 'Nevada':'NV', 'New Mexico':'NM', 'North Dakota':'ND', 'Oklahoma':'OK', 
                    'Oregon':'OR', 'South Dakota':'SD', 'Texas':'TX', 'Utah':'UT', 'Washington':'WA', 'Wyoming':'WY'}
# for year in [2018, 2020, 2024, 2030, 2040, 2050]:
for year in [2018, 2030, 2040]:
    efs_loads['WECC_'+str(year)+'_LDV'] = 0
    efs_loads['WECC_'+str(year)+'_nonLDV'] = 0
    for state in states:
        frac = wecc_regions.loc[(wecc_regions['State_name']==state)]['Population'].sum() / regions1.loc[regions1['State_name']==state]['Population'].sum()
        efs_loads['WECC_'+str(year)+'_LDV'] += frac*efs_loads[state_shortforms[state]+'_'+str(year)+'_LDV'].values
        efs_loads['WECC_'+str(year)+'_nonLDV'] += frac*efs_loads[state_shortforms[state]+'_'+str(year)+'_nonLDV'].values


# extract scaling factor: 

In [19]:
print((efs_loads['WECC_2030_nonLDV'] / efs_loads['WECC_2018_nonLDV']).mean())
print((efs_loads['WECC_2030_nonLDV'] / efs_loads['WECC_2018_nonLDV']).median())

1.1031437192139566
1.104138497781564


In [20]:
print((efs_loads['WECC_2040_nonLDV'] / efs_loads['WECC_2018_nonLDV']).mean())
print((efs_loads['WECC_2040_nonLDV'] / efs_loads['WECC_2018_nonLDV']).median())

1.2135844334985646
1.2150301835420632


In [21]:
efs_loads['WECC_2035_nonLDV'] = (1/2)*(efs_loads['WECC_2030_nonLDV'] + efs_loads['WECC_2040_nonLDV'])

In [22]:
print((efs_loads['WECC_2035_nonLDV'] / efs_loads['WECC_2018_nonLDV']).mean())
print((efs_loads['WECC_2035_nonLDV'] / efs_loads['WECC_2018_nonLDV']).median())

1.1583640763562568
1.1595842973226727
